In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
train_path = '/content/drive/My Drive/digit-recognizer/train.csv'
train_data = pd.read_csv(train_path)

test_path = '/content/drive/My Drive/digit-recognizer/test.csv'
test_data = pd.read_csv(test_path)

In [4]:
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [7]:
train_data.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [8]:
y_train = train_data['label'].to_numpy()
print(type(y_train))
print(y_train.shape)
print(type(train_data['label']))

<class 'numpy.ndarray'>
(42000,)
<class 'pandas.core.series.Series'>


In [9]:
x_train = train_data.iloc[:, 1:].to_numpy()
print(x_train.shape)
print(type(x_train))

(42000, 784)
<class 'numpy.ndarray'>


In [10]:
x_tr = []
for i in range(len(x_train)):
  x_tr.append(x_train[i].reshape(28,28))
X_train = np.array(x_tr)

In [13]:
x_test = test_data.iloc[:, :].to_numpy()
x_te = []
for i in range(len(x_test)):
  x_te.append(x_test[i].reshape(28, 28))
X_test = np.array(x_te)


In [14]:
X_test.shape

(28000, 28, 28)

In [15]:
X_train.shape

(42000, 28, 28)

In [16]:
X_train = X_train.reshape(42000, 28, 28, 1)
X_test = X_test.reshape(28000, 28, 28, 1)

In [17]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [21]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, eopch, logs={}):
    if logs.get('loss') <= 0.01:
      print("\nReached Target Loss Accuraccy,  Cancelling Training!!!\n")
      self.model.stop_training=True


def train_model():

  callback = MyCallback()
  model = tf.keras.models.Sequential()

  model = tf.keras.models.Sequential([
                                      
      tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(2,2),

      tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),

      tf.keras.layers.Flatten(), 
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer = tf.optimizers.Adam(), 
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  model.fit(X_train, y_train, epochs = 20, callbacks = [callback])

  return model


In [22]:
model = train_model()

Epoch 1/20
1313/1313 [==============================] - 7s 6ms/step - loss: 0.1267 - accuracy: 0.9602
Epoch 2/20
1313/1313 [==============================] - 7s 6ms/step - loss: 0.0410 - accuracy: 0.9866
Epoch 3/20
1313/1313 [==============================] - 7s 6ms/step - loss: 0.0265 - accuracy: 0.9920
Epoch 4/20
1313/1313 [==============================] - 7s 6ms/step - loss: 0.0188 - accuracy: 0.9938
Epoch 5/20
1313/1313 [==============================] - 7s 6ms/step - loss: 0.0156 - accuracy: 0.9949
Epoch 6/20
1313/1313 [==============================] - 7s 6ms/step - loss: 0.0119 - accuracy: 0.9960
Epoch 7/20
1313/1313 [==============================] - 7s 6ms/step - loss: 0.0107 - accuracy: 0.9964
Epoch 8/20
1305/1313 [============================>.] - ETA: 0s - loss: 0.0063 - accuracy: 0.9983
Reached Target Loss Accuraccy,  Cancelling Training!!!

1313/1313 [==============================] - 7s 6ms/step - loss: 0.0063 - accuracy: 0.9983


In [23]:
pred = model.predict(X_test)

In [24]:
pred[0]

array([2.9743198e-15, 3.0533880e-17, 1.0000000e+00, 2.7910110e-14,
       1.3626843e-10, 9.7236276e-23, 3.1014943e-18, 2.1354828e-13,
       3.0286029e-14, 1.2250153e-15], dtype=float32)

In [25]:
predictions=[]
for i in range(len(pred)):
  y = pred[i].tolist()
  predictions.append([i+1, y.index(max(y))])

In [26]:
predictions[-1]

[28000, 2]

In [27]:
predictions=np.array(predictions)
print(predictions.shape)
print(type(predictions))

(28000, 2)
<class 'numpy.ndarray'>


In [28]:
sub = pd.DataFrame(data=predictions, columns = ['ImageId', 'Label'])
sub.head(20)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [29]:
sub.to_csv('MNIST Predictions.csv', index=False)